# Linearization

### 0. Imports

In [1]:
import numpy as np
import sympy as sym
from scipy import linalg
from scipy.interpolate import interp1d
from IPython.display import display, IFrame, HTML

# 1. Derive models

## 1.1 Define symbolic variables

Define states.

In [2]:
# position (m)
o_x, o_y, o_z = sym.symbols('o_x, o_y, o_z')

# orientation angles (rad)
psi, theta, phi = sym.symbols('psi, theta, phi')

# linear velocity (m/s)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')

Define inputs.

In [3]:
# gyroscope measurements - components of angular velocity (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# z-axis accelerometer measurement - specific force (meters / second^2)
a_z = sym.symbols('a_z')

# 
tau_x, tau_y, tau_z, f_z = sym.symbols("tau_x, tau_y, tau_z, f_z")

Define outputs.

In [4]:
d0, d1, d2, d3, d4, d5, d6, d7, n_x, n_y, r = sym.symbols('d0, d1, d2, d3, d4, d5, d6, d7,n_x, n_y, r')
a_0, a_1, a_2, a_3, a_4, a_5, a_6, a_7 = sym.symbols('a_0, a_1, a_2, a_3, a_4, a_5, a_6, a_7 ')

Define parameters.

In [5]:
g, k_flow = sym.symbols('g, k_flow')

Create linear and angular velocity vectors (in coordinates of the body frame).

In [6]:
v_01in1 = sym.Matrix([[v_x], [v_y], [v_z]])
w_01in1 = sym.Matrix([[w_x], [w_y], [w_z]])

In [7]:
s = [o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z]
i = [w_x, w_y, w_z, a_z]

## 1.2 Define kinematics of orientation

### 1.2.1 Rotation matrix in terms of yaw, pitch, roll angles

Define individual rotation matrices.

In [8]:
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi), sym.cos(psi), 0],
                 [0, 0, 1]])

Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)],
                 [0, 1, 0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])

Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi), sym.cos(phi)]])

Apply sequential transformation.

In [9]:
R_1in0 = Rz * Ry * Rx

### 1.2.2 Map from angular velocity to angular rates

In [10]:
Ninv = sym.Matrix.hstack((Ry * Rx).T * sym.Matrix([[0], [0], [1]]),
                              (Rx).T * sym.Matrix([[0], [1], [0]]),
                                       sym.Matrix([[1], [0], [0]]))

Compute $N$.

In [11]:
N = sym.simplify(Ninv.inv())

## 1.3 Derive equations of motion

Ratio of net thrust to mass in terms of z-axis accelerometer measurement.

In [12]:
f_z_over_m = a_z + (w_01in1.cross(v_01in1))[2]

Ratio of forces to mass.

In [13]:
f_in1_over_m = R_1in0.T * sym.Matrix([[0], [0], [-g]]) + sym.Matrix([[0], [0], [f_z_over_m]])

Equations of motion.

In [14]:
f = sym.Matrix.vstack(
    R_1in0 * v_01in1,
    N * w_01in1,
    (f_in1_over_m - w_01in1.cross(v_01in1)),
)

Show equations of motion, which have the form

$$\dot{s} = f(s, i, p)$$

where

$$
s = \begin{bmatrix} o_x \\ o_y \\ o_z \\ \psi \\ \theta \\ \phi \\ v_x \\ v_y \\ v_z \end{bmatrix}
\qquad\qquad
i = \begin{bmatrix} w_x \\ w_y \\ w_z \\ a_z \end{bmatrix}
\qquad\qquad
p = \begin{bmatrix} g \\ k_\text{flow} \end{bmatrix}.
$$

In [15]:
f

Matrix([
[ v_x*cos(psi)*cos(theta) + v_y*(sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi)) + v_z*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi))],
[v_x*sin(psi)*cos(theta) + v_y*(sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi)) + v_z*(-sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi))],
[                                                                       -v_x*sin(theta) + v_y*sin(phi)*cos(theta) + v_z*cos(phi)*cos(theta)],
[                                                                                         w_y*sin(phi)/cos(theta) + w_z*cos(phi)/cos(theta)],
[                                                                                                               w_y*cos(phi) - w_z*sin(phi)],
[                                                                                   w_x + w_y*sin(phi)*tan(theta) + w_z*cos(phi)*tan(theta)],
[                                                                                                          g*sin(theta) + v_y*w_z - v_z*w_y

# 2. Observer Equations with Loco Positioning

## 2.1 Loco Positioning Variables and Equations

Define position coordinates of the 8 loco positioning nodes.

In [16]:
x_0, y_0, z_0 = sym.symbols("x_0, y_0, z_0")
x_1, y_1, z_1 = sym.symbols("x_1, y_1, z_1")
x_2, y_2, z_2 = sym.symbols("x_2, y_2, z_2")
x_3, y_3, z_3 = sym.symbols("x_3, y_3, z_3")
x_4, y_4, z_4 = sym.symbols("x_4, y_4, z_4")
x_5, y_5, z_5 = sym.symbols("x_5, y_5, z_5")
x_6, y_6, z_6 = sym.symbols("x_6, y_6, z_6")
x_7, y_7, z_7 = sym.symbols("x_7, y_7, z_7")

Define distance output variables for each node.

In [17]:
d_0, d_1, d_2, d_3, d_4, d_5, d_6, d_7 = sym.symbols('d_0, d_1, d_2, d_3, d_4, d_5, d_6, d_7')

Define equations representing the distance to each node from the drone's position.

$d_i = \sqrt{(o_x - x_i)^2 + (o_y - y_i)^2 + (z_x - z_i)^2}$

In [18]:
anchor_dist_0 = sym.sqrt((o_x - x_0)**2 + (o_y - y_0)**2 + (o_z - z_0)**2)
anchor_dist_1 = sym.sqrt((o_x - x_1)**2 + (o_y - y_1)**2 + (o_z - z_1)**2)
anchor_dist_2 = sym.sqrt((o_x - x_2)**2 + (o_y - y_2)**2 + (o_z - z_2)**2)
anchor_dist_3 = sym.sqrt((o_x - x_3)**2 + (o_y - y_3)**2 + (o_z - z_3)**2)
anchor_dist_4 = sym.sqrt((o_x - x_4)**2 + (o_y - y_4)**2 + (o_z - z_4)**2)
anchor_dist_5 = sym.sqrt((o_x - x_5)**2 + (o_y - y_5)**2 + (o_z - z_5)**2)
anchor_dist_6 = sym.sqrt((o_x - x_6)**2 + (o_y - y_6)**2 + (o_z - z_6)**2)
anchor_dist_7 = sym.sqrt((o_x - x_7)**2 + (o_y - y_7)**2 + (o_z - z_7)**2)

Show measurement equations, which have the form

$$o = h(s, i, p)$$

where

$$
o = \begin{bmatrix} n_x \\ n_y \\ r \end{bmatrix}
\qquad\qquad
s = \begin{bmatrix} o_x \\ o_y \\ o_z \\ \psi \\ \theta \\ \phi \\ v_x \\ v_y \\ v_z \end{bmatrix}
\qquad\qquad
i = \begin{bmatrix} w_x \\ w_y \\ w_z \\ a_z \end{bmatrix}
\qquad\qquad
p = \begin{bmatrix} g \\ k_\text{flow} \end{bmatrix}.
$$

Observer equation, $h$.

In [51]:
h = sym.Matrix([
    anchor_dist_0, # d_0
    anchor_dist_1, # d_1
    anchor_dist_2, # d_2
    anchor_dist_3, # d_3
    anchor_dist_4, # d_4
    anchor_dist_5, # d_5
    anchor_dist_6, # d_6
    anchor_dist_7, # d_7
    k_flow * (v_x - o_z * w_y) / o_z, # n_x
    k_flow * (v_y + o_z * w_x) / o_z, # n_y
    o_z / (sym.cos(phi) * sym.cos(theta)) # r
])

Show $h$.

In [20]:
h

Matrix([
[sqrt((o_x - x_0)**2 + (o_y - y_0)**2 + (o_z - z_0)**2)],
[sqrt((o_x - x_1)**2 + (o_y - y_1)**2 + (o_z - z_1)**2)],
[sqrt((o_x - x_2)**2 + (o_y - y_2)**2 + (o_z - z_2)**2)],
[sqrt((o_x - x_3)**2 + (o_y - y_3)**2 + (o_z - z_3)**2)],
[sqrt((o_x - x_4)**2 + (o_y - y_4)**2 + (o_z - z_4)**2)],
[sqrt((o_x - x_5)**2 + (o_y - y_5)**2 + (o_z - z_5)**2)],
[sqrt((o_x - x_6)**2 + (o_y - y_6)**2 + (o_z - z_6)**2)],
[sqrt((o_x - x_7)**2 + (o_y - y_7)**2 + (o_z - z_7)**2)],
[                           k_flow*(-o_z*w_y + v_x)/o_z],
[                            k_flow*(o_z*w_x + v_y)/o_z],
[                             o_z/(cos(phi)*cos(theta))]])

## 2.2 Output

Define original measurement variables.

In [21]:
n_x, n_y, r = sym.symbols("n_x, n_y, r")

Define original system with new outputs.

In [22]:
s = [o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z]
i = [w_x, w_y, w_z, a_z]
o = [d_0, d_1, d_2, d_3,
     d_4, d_5, d_6, d_7,
     n_x, n_y, r]
p = [g, k_flow]

Define equilibrium point variables.

In [23]:
o_x_eq = sym.symbols('o_x_eq')
o_y_eq = sym.symbols('o_y_eq')
o_z_eq = sym.symbols('o_z_eq')
s_eq = [o_x_eq, o_y_eq, o_z_eq, 0, 0, 0, 0, 0, 0]
i_eq = [0, 0, 0, g]
s_eq = [sym.nsimplify(a) for a in s_eq]
i_eq = [sym.nsimplify(a) for a in i_eq]

Define state, input, and output vectors.

In [24]:
x = sym.Matrix([o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z]) - sym.Matrix(s_eq)
u = sym.Matrix([w_x, w_y, w_z, a_z]) - sym.Matrix(i_eq)
y = sym.Matrix(o) - h.subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))

Show $y$.

In [25]:
y

Matrix([
[d_0 - sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2)],
[d_1 - sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2)],
[d_2 - sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2)],
[d_3 - sqrt((o_x_eq - x_3)**2 + (o_y_eq - y_3)**2 + (o_z_eq - z_3)**2)],
[d_4 - sqrt((o_x_eq - x_4)**2 + (o_y_eq - y_4)**2 + (o_z_eq - z_4)**2)],
[d_5 - sqrt((o_x_eq - x_5)**2 + (o_y_eq - y_5)**2 + (o_z_eq - z_5)**2)],
[d_6 - sqrt((o_x_eq - x_6)**2 + (o_y_eq - y_6)**2 + (o_z_eq - z_6)**2)],
[d_7 - sqrt((o_x_eq - x_7)**2 + (o_y_eq - y_7)**2 + (o_z_eq - z_7)**2)],
[                                                                  n_x],
[                                                                  n_y],
[                                                          -o_z_eq + r]])

# 3. Linearization of the System

## 3.1 Original System

Find $A$, $B$, $C$, and $D$ ignoring observability.

In [26]:
A = f.jacobian(s).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))
B = f.jacobian(i).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))
C = h.jacobian(s).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))
D = h.jacobian(i).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))

## 3.2 Observable System

Focus on the observable elements of the system.

In [27]:
s_obs_index = [0, 1, 2, 4, 5, 6, 7, 8]
A_obs = A[s_obs_index, :][:, s_obs_index]
B_obs = B[s_obs_index, :]
C_obs = C[:, s_obs_index]
D_obs = D
x_obs = sym.Matrix([o_x - o_x_eq, o_y - o_y_eq, o_z - o_z_eq, theta, phi, v_x, v_y, v_z])

Show observable versions of the matrices.

In [28]:
A_obs

Matrix([
[0, 0, 0, 0,  0, 1, 0, 0],
[0, 0, 0, 0,  0, 0, 1, 0],
[0, 0, 0, 0,  0, 0, 0, 1],
[0, 0, 0, 0,  0, 0, 0, 0],
[0, 0, 0, 0,  0, 0, 0, 0],
[0, 0, 0, g,  0, 0, 0, 0],
[0, 0, 0, 0, -g, 0, 0, 0],
[0, 0, 0, 0,  0, 0, 0, 0]])

In [29]:
B_obs

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 1, 0, 0],
[1, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 1]])

In [30]:
C_obs

Matrix([
[(o_x_eq - x_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2), (o_y_eq - y_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2), (o_z_eq - z_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2), 0, 0,             0,             0, 0],
[(o_x_eq - x_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2), (o_y_eq - y_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2), (o_z_eq - z_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2), 0, 0,             0,             0, 0],
[(o_x_eq - x_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2), (o_y_eq - y_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2), (o_z_eq - z_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2), 0, 0,             0,             0, 0],
[(o_x_eq - x_3)/sqrt((o_x_eq - x_3)**2 + (o_y_eq - y_3)**2 + (o_z_eq - z_3)**2), (o_y_eq - y_3)/sqrt((o_x_eq - x_3)**2 + (o_y_eq - y_3)**2 + (o_z_eq

In [31]:
D_obs

Matrix([
[     0,       0, 0, 0],
[     0,       0, 0, 0],
[     0,       0, 0, 0],
[     0,       0, 0, 0],
[     0,       0, 0, 0],
[     0,       0, 0, 0],
[     0,       0, 0, 0],
[     0,       0, 0, 0],
[     0, -k_flow, 0, 0],
[k_flow,       0, 0, 0],
[     0,       0, 0, 0]])

## 3.3 Symbolic Representation

### 3.3.1 Measurement Error

Symbolically calculate the error in the measurements.

In [32]:
C_obs * x_obs + D_obs * u - y

Matrix([
[-d_0 + (o_x - o_x_eq)*(o_x_eq - x_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) + (o_y - o_y_eq)*(o_y_eq - y_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) + (o_z - o_z_eq)*(o_z_eq - z_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) + sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2)],
[-d_1 + (o_x - o_x_eq)*(o_x_eq - x_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + (o_y - o_y_eq)*(o_y_eq - y_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + (o_z - o_z_eq)*(o_z_eq - z_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2)],
[-d_2 + (o_x - o_x_eq)*(o_x_eq - x_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2) + (o_y - o_y_eq)*(o_y_eq - y_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2) + (o_z - o_z_eq)*(o_z_eq - z_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_

### 3.3.2 State Estimate Updating

Define variables for each element of $L$.

In [33]:
L_0, L_1, L_2, L_3, L_4, L_5, L_6, L_7, L_8, L_9, L_10 = sym.symbols('L_0, L_1, L_2, L_3, L_4, L_5, L_6, L_7, L_8, L_9, L_10')
L_11, L_12, L_13, L_14, L_15, L_16, L_17, L_18, L_19, L_20, L_21 = sym.symbols('L_11, L_12, L_13, L_14, L_15, L_16, L_17, L_18, L_19, L_20, L_21')
L_22, L_23, L_24, L_25, L_26, L_27, L_28, L_29, L_30, L_31, L_32 = sym.symbols('L_22, L_23, L_24, L_25, L_26, L_27, L_28, L_29, L_30, L_31, L_32')
L_33, L_34, L_35, L_36, L_37, L_38, L_39, L_40, L_41, L_42, L_43 = sym.symbols('L_33, L_34, L_35, L_36, L_37, L_38, L_39, L_40, L_41, L_42, L_43')
L_44, L_45, L_46, L_47, L_48, L_49, L_50, L_51, L_52, L_53, L_54 = sym.symbols('L_44, L_45, L_46, L_47, L_48, L_49, L_50, L_51, L_52, L_53, L_54')
L_55, L_56, L_57, L_58, L_59, L_60, L_61, L_62, L_63, L_64, L_65 = sym.symbols('L_55, L_56, L_57, L_58, L_59, L_60, L_61, L_62, L_63, L_64, L_65')
L_66, L_67, L_68, L_69, L_70, L_71, L_72, L_73, L_74, L_75, L_76 = sym.symbols('L_66, L_67, L_68, L_69, L_70, L_71, L_72, L_73, L_74, L_75, L_76')
L_77, L_78, L_79, L_80, L_81, L_82, L_83, L_84, L_85, L_86, L_87 = sym.symbols('L_77, L_78, L_79, L_80, L_81, L_82, L_83, L_84, L_85, L_86, L_87')

Define $L$.

In [34]:
L_var_sym = sym.Matrix([[L_0, L_1, L_2, L_3, L_4, L_5, L_6, L_7, L_8, L_9, L_10],
                        [L_11, L_12, L_13, L_14, L_15, L_16, L_17, L_18, L_19, L_20, L_21],
                        [L_22, L_23, L_24, L_25, L_26, L_27, L_28, L_29, L_30, L_31, L_32],
                        [L_33, L_34, L_35, L_36, L_37, L_38, L_39, L_40, L_41, L_42, L_43],
                        [L_44, L_45, L_46, L_47, L_48, L_49, L_50, L_51, L_52, L_53, L_54],
                        [L_55, L_56, L_57, L_58, L_59, L_60, L_61, L_62, L_63, L_64, L_65],
                        [L_66, L_67, L_68, L_69, L_70, L_71, L_72, L_73, L_74, L_75, L_76],
                        [L_77, L_78, L_79, L_80, L_81, L_82, L_83, L_84, L_85, L_86, L_87]])

Find symbolic representation of state estimate equation.

In [35]:
x_obs_dot = A_obs * x_obs + B_obs * u
y_err_sym = C_obs * x_obs + D_obs * u - y
x_hat_dot_sym = x_obs_dot - L_var_sym * y_err_sym

Show $\dot{\hat{x}}$.

In [36]:
x_hat_dot_sym

Matrix([
[              -L_0*(-d_0 + (o_x - o_x_eq)*(o_x_eq - x_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) + (o_y - o_y_eq)*(o_y_eq - y_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) + (o_z - o_z_eq)*(o_z_eq - z_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) + sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2)) - L_1*(-d_1 + (o_x - o_x_eq)*(o_x_eq - x_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + (o_y - o_y_eq)*(o_y_eq - y_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + (o_z - o_z_eq)*(o_z_eq - z_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2)) - L_10*(o_z - r) - L_2*(-d_2 + (o_x - o_x_eq)*(o_x_eq - x_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2) + (o_y - o_y_eq)*(o_y_eq - y_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2) + (o_z - o_z_eq)*(o_z_

## 3.4 Numeric Implementation

### 3.4.1 Numeric Matrices

Define anchor positions literally.

In [37]:
anchors_x_sym = [x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7]
anchors_y_sym = [y_0, y_1, y_2, y_3, y_4, y_5, y_6, y_7]
anchors_z_sym = [z_0, z_1, z_2, z_3, z_4, z_5, z_6, z_7]
anchors_x = [-1.82, -3.57, 1.82, 2.67, -3.61, 1.84, 2.74, -1.85]
anchors_y = [2.18, 3.18, 2.21, -3.30, -3.30, -0.90, 3.18, -0.93]
anchors_z = [1.10, 2.24, 1.10, 2.20, 2.25, 1.10, 2.24, 1.09]

Find numeric $A$, $B$, and $C$.

In [38]:
A_arr = np.array(A_obs.subs(g, -9.81).tolist(), dtype=np.float64)
B_arr = np.array(B_obs.subs(g, -9.81).tolist(), dtype=np.float64)
C_arr = np.array(C_obs.subs(tuple(zip(anchors_x_sym, anchors_x))).subs(tuple(zip(anchors_y_sym, anchors_y))).subs(tuple(zip(anchors_z_sym, anchors_z))).subs(o_x_eq, 0).subs(o_y_eq, 0).subs(o_z_eq, 0.5).subs(k_flow, 0.01 * 30.0 / np.deg2rad(4.2)).tolist(), dtype=np.float64)

### 3.3.2 State Estimate Updating

Define a function to use LQR.

In [39]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K

Find $L$ using LQR with identity weights.

In [40]:
Q = np.diag([
    1,             # d_0
    1,             # d_1
    1,             # d_2
    1,             # d_3
    1,             # d_4
    1,             # d_5
    1,             # d_6
    1,             # d_7
    1,             # n_x
    1,             # n_y
    1,             # r
])

R = np.diag([
    1,             # o_x
    1,             # o_y
    1,             # o_z
    1,             # theta
    1,             # phi
    1,             # v_x
    1,             # v_y
    1,             # v_z
])

L_identity = lqr(np.array(A_arr.tolist()).T, np.array(C_arr.tolist()).T, linalg.inv(R), linalg.inv(Q)).T

Find L using LQR with custom weights.

In [41]:
Q = np.diag([
    1/0.049**2,             # d_0
    1/0.072**2,             # d_1
    1/0.046**2,             # d_2
    1/0.114**2,             # d_3
    1/0.063**2,             # d_4
    1/0.048**2,             # d_5
    1/0.070**2,             # d_6
    1/0.038**2,             # d_7
    1/1.696**2,             # n_x
    1/1.738**2,             # n_y
    1/0.148**2,             # r
])

R = np.diag([
    1/0.094**2,             # o_x
    1/0.111**2,             # o_y
    1/0.009**2,             # o_z
    1/0.036**2,             # theta
    1/0.030**2,             # phi
    1/0.176**2,             # v_x
    1/0.178**2,             # v_y
    1/0.354**2,             # v_z
])

L_custom = lqr(np.array(A_arr.tolist()).T, np.array(C_arr.tolist()).T, linalg.inv(R), linalg.inv(Q)).T

Choose which $L$ to use in the following cell.

In [42]:
custom = True               # Set to True to use custom L

if custom:
    L = L_custom
else:
    L = L_identity

Find numeric representation of the state estimate equation.

In [43]:
A_sym = sym.Matrix(A)
B_sym = sym.Matrix(B)
L_sym = sym.Matrix(L)
x_dot_sym = A_sym * x + B_sym * u
x_obs_dot_sym = A_obs * x_obs + B_obs * u
y_err_sym = C_obs * x_obs + D_obs * u - y
x_hat_dot_sym = x_obs_dot_sym - L_sym * y_err_sym

Show $\dot{\hat{x}}$.

In [44]:
x_hat_dot_sym

Matrix([
[                                                   1.12707648705855*d_0 + 0.5476728004585*d_1 - 1.06158812414135*d_2 - 0.227059392530423*d_3 + 0.552105984522811*d_4 - 1.66587269822643*d_5 - 0.475526854975272*d_6 + 2.07114365067007*d_7 + 0.00168407966994269*k_flow*w_x + 0.0214688706014934*k_flow*w_y - 0.0214688706014934*k_flow*v_x/o_z_eq + 0.00168407966994269*k_flow*v_y/o_z_eq + 0.0214688706014934*n_x - 0.00168407966994269*n_y - 0.0339666169208879*o_z + 0.0339666169208879*r + v_x - 1.12707648705855*(o_x - o_x_eq)*(o_x_eq - x_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) - 0.5476728004585*(o_x - o_x_eq)*(o_x_eq - x_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + 1.06158812414135*(o_x - o_x_eq)*(o_x_eq - x_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2) + 0.227059392530423*(o_x - o_x_eq)*(o_x_eq - x_3)/sqrt((o_x_eq - x_3)**2 + (o_y_eq - y_3)**2 + (o_z_eq - z_3)**2) - 0.552105984522811*(o_x - o_x_eq)*(o_x_eq - x_4)/sqrt

Introduce $Y_{\text{err}}$ matrix to simplify equations for coding.

In [45]:
Y_0, Y_1, Y_2, Y_3, Y_4, Y_5, Y_6, Y_7, Y_8, Y_9, Y_10 = sym.symbols('Y_0, Y_1, Y_2, Y_3, Y_4, Y_5, Y_6, Y_7, Y_8, Y_9, Y_10')
Y_err = sym.Matrix([Y_0, Y_1, Y_2, Y_3, Y_4, Y_5, Y_6, Y_7, Y_8, Y_9, Y_10])
output_list = (x_obs_dot_sym - L_sym * Y_err).tolist()

Format equations for C code.

In [46]:
for i in range(len(output_list)):
    line = str(output_list[i])
    output_list[i] = line.replace("*", "f*" )
    if i == 5 or i == 6:
        output_list[i] = line.replace("gf*", "g*" )
    print(output_list[i])
    print("\n")

[-1.12707648705855f*Y_0 - 0.5476728004585f*Y_1 - 0.0339666169208879f*Y_10 + 1.06158812414135f*Y_2 + 0.227059392530423f*Y_3 - 0.552105984522811f*Y_4 + 1.66587269822643f*Y_5 + 0.475526854975272f*Y_6 - 2.07114365067007f*Y_7 - 0.0214688706014934f*Y_8 + 0.00168407966994269f*Y_9 + v_x]


[1.73816242606271f*Y_0 + 0.674035871250208f*Y_1 + 0.0182697597468506f*Y_10 + 1.69091746248964f*Y_2 - 0.325418193645903f*Y_3 - 0.787896464809938f*Y_4 - 1.1956767991533f*Y_5 + 0.657924407506712f*Y_6 - 1.35219734302355f*Y_7 + 0.00197779925500575f*Y_8 - 0.0227901005026907f*Y_9 + v_y]


[0.817938135344624f*Y_0 + 0.722615443998249f*Y_1 - 0.603140474287141f*Y_10 + 1.35653032358065f*Y_2 + 0.433584324169403f*Y_3 + 1.05513109876531f*Y_4 + 1.96351857478474f*Y_5 + 1.06683224932403f*Y_6 + 2.18412437166829f*Y_7 - 0.00242935847370092f*Y_8 + 0.00108821463471846f*Y_9 + v_z]


[0.196479899076098f*Y_0 + 0.0985626400158723f*Y_1 + 0.0023957978163299f*Y_10 - 0.194663521002678f*Y_2 - 0.0361702630754423f*Y_3 + 0.111154854141702f*Y_